In [4]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

## s*10 (-2.5)

In [10]:
k = 5
s = -1/(k-1)*10
C_num = 3
rep = 40

In [11]:
s

-2.5

In [12]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [13]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [14]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2/'
    file_data = rountine + 'all_data_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
0.8366666666666667
[[23.  0.  0. 14. 11.]
 [ 0. 38.  3.  0.  0.]
 [ 0. 13. 73.  0.  0.]
 [ 0.  8.  0. 51.  0.]
 [ 0.  0.  0.  0. 66.]]
0.83
[[16.  0.  0. 17.  8.]
 [ 0. 42.  1.  0.  0.]
 [ 0. 10. 89.  0.  0.]
 [ 0. 15.  0. 41.  0.]
 [ 0.  0.  0.  0. 61.]]
2
0.87
[[14.  0.  0. 12.  9.]
 [ 0. 40.  0.  0.  0.]
 [ 0.  6. 84.  0.  0.]
 [ 0. 11.  0. 64.  0.]
 [ 1.  0.  0.  0. 59.]]
0.8333333333333334
[[17.  0.  0. 15. 10.]
 [ 0. 59.  0.  0.  0.]
 [ 0.  8. 58.  0.  0.]
 [ 0. 17.  0. 50.  0.]
 [ 0.  0.  0.  0. 66.]]
3
0.8466666666666667
[[19.  0.  0. 11. 11.]
 [ 0. 59.  0.  0.  0.]
 [ 0. 10. 71.  0.  0.]
 [ 0. 11.  0. 40.  0.]
 [ 3.  0.  0.  0. 65.]]
0.8333333333333334
[[15.  0.  0. 12.  6.]
 [ 0. 54.  0.  0.  0.]
 [ 0. 12. 73.  0.  0.]
 [ 1. 15.  0. 43.  0.]
 [ 4.  0.  0.  0. 65.]]
4
0.85
[[37.  0.  0.  8. 10.]
 [ 0. 44.  0.  0.  0.]
 [ 0.  8. 71.  0.  0.]
 [ 2. 15.  0. 53.  0.]
 [ 2.  0.  0.  0. 50.]]
0.85
[[23.  0.  0.  9.  6.]
 [ 0. 45.  1.  0.  0.]
 [ 0. 17. 68.  0.  0.]
 [ 1. 11.  0. 5

0.8666666666666667
[[22.  0.  0. 10.  9.]
 [ 0. 44.  0.  0.  0.]
 [ 0. 10. 63.  0.  0.]
 [ 0. 10.  0. 54.  0.]
 [ 1.  0.  0.  0. 77.]]
0.8166666666666667
[[23.  0.  0. 19. 10.]
 [ 0. 48.  0.  0.  0.]
 [ 0. 10. 58.  0.  0.]
 [ 1. 15.  0. 53.  0.]
 [ 1.  0.  0.  0. 62.]]
36
0.8333333333333334
[[ 4.  0.  0. 20. 11.]
 [ 0. 52.  0.  0.  0.]
 [ 0.  7. 88.  0.  0.]
 [ 0. 12.  0. 53.  0.]
 [ 0.  0.  0.  0. 53.]]
0.8366666666666667
[[12.  0.  0. 19.  8.]
 [ 0. 50.  2.  1.  0.]
 [ 0.  9. 72.  0.  0.]
 [ 0. 10.  0. 52.  0.]
 [ 0.  0.  0.  0. 65.]]
37
0.8533333333333334
[[16.  0.  0. 14.  7.]
 [ 0. 53.  0.  0.  0.]
 [ 0. 12. 74.  0.  0.]
 [ 0. 11.  0. 39.  0.]
 [ 0.  0.  0.  0. 74.]]
0.86
[[20.  0.  0.  9. 10.]
 [ 0. 36.  0.  0.  0.]
 [ 0. 11. 77.  0.  0.]
 [ 0. 14.  0. 63.  0.]
 [ 0.  0.  0.  0. 60.]]
38
0.85
[[12.  0.  0. 18.  6.]
 [ 0. 54.  1.  0.  0.]
 [ 0. 11. 80.  0.  0.]
 [ 0.  9.  0. 49.  0.]
 [ 0.  0.  0.  0. 60.]]
0.8433333333333334
[[12.  0.  0. 18. 10.]
 [ 0. 52.  0.  0.  0.]
 [ 0.  9.

In [15]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.83, 0.8333333333333334, 0.8333333333333334, 0.85, 0.83, 0.8, 0.8433333333333334, 0.81, 0.84, 0.8166666666666667, 0.81, 0.8333333333333334, 0.9133333333333333, 0.8933333333333333, 0.8366666666666667, 0.8066666666666666, 0.8466666666666667, 0.7833333333333333, 0.8066666666666666, 0.8033333333333333, 0.8266666666666667, 0.8666666666666667, 0.8633333333333333, 0.8433333333333334, 0.8666666666666667, 0.8466666666666667, 0.83, 0.8533333333333334, 0.81, 0.8233333333333334, 0.83, 0.8166666666666667, 0.8366666666666667, 0.86, 0.8433333333333334, 0.8666666666666667, 0.82]
0.8357657657657658
0.025792526895068304
[0.7849197196565618, 0.804816620989547, 0.7946091524488317, 0.8345044592836055, 0.789578256276098, 0.7542385356094634, 0.7858312339989759, 0.7281433800117935, 0.8056104102615731, 0.7967235108134963, 0.7595197032106437, 0.7533657122113476, 0.8872566157522049, 0.8762137053351446, 0.7838680939948859, 0.7563408005900871, 0.8225321902802234, 0.6999362275948892, 0.7084070175438597, 0.7600126

### s5

In [16]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5/'
    file_data = rountine + 'all_data_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.8433333333333334
[[71.  3.  0.  4.  5.]
 [ 1. 41.  5.  1.  0.]
 [ 1.  3. 71.  1.  4.]
 [ 4.  1.  3. 42.  0.]
 [ 9.  0.  2.  0. 28.]]
0.8266666666666667
[[34.  1.  0. 10.  2.]
 [ 1. 36.  5.  1.  0.]
 [ 2.  6. 76.  2.  3.]
 [ 2.  5.  1. 51.  3.]
 [ 4.  0.  3.  1. 51.]]
2
0.8633333333333333
[[59.  1.  0.  5.  5.]
 [ 0. 36.  4.  0.  0.]
 [ 1.  1. 69.  3.  5.]
 [ 5.  1.  1. 49.  3.]
 [ 3.  0.  3.  0. 46.]]
0.87
[[53.  1.  0.  3.  3.]
 [ 2. 39. 11.  0.  0.]
 [ 0.  0. 84.  1.  3.]
 [ 4.  3.  1. 52.  4.]
 [ 3.  0.  0.  0. 33.]]
3
0.8466666666666667
[[41.  6.  0.  6.  3.]
 [ 4. 47.  4.  1.  0.]
 [ 0.  2. 84.  1.  3.]
 [ 0.  4.  3. 49.  2.]
 [ 6.  0.  1.  0. 33.]]
0.84
[[42.  1.  0.  8.  0.]
 [ 2. 34.  5.  0.  0.]
 [ 2.  5. 96.  7.  5.]
 [ 0.  1.  1. 52.  1.]
 [10.  0.  0.  0. 28.]]
4
0.8466666666666667
[[50.  1.  0.  9.  2.]
 [ 0. 38.  9.  0.  0.]
 [ 0.  5. 79.  3.  2.]
 [ 3.  3.  5. 50.  2.]
 [ 2.  0.  0.  0. 37.]]
0.8566666666666667
[[46.  0.  0. 10.  1.]
 [ 2. 36.  4.  0.  0.]
 [ 0.  3. 

0.86
[[53.  3.  0.  4.  0.]
 [ 1. 33. 10.  0.  1.]
 [ 0.  0. 76.  0.  4.]
 [ 4.  3.  2. 53.  2.]
 [ 8.  0.  0.  0. 43.]]
0.8166666666666667
[[40.  9.  0.  6.  4.]
 [ 2. 37. 11.  0.  0.]
 [ 0.  0. 69.  0.  2.]
 [ 7.  4.  6. 47.  1.]
 [ 5.  0.  0.  0. 50.]]
34
0.86
[[59.  6.  0.  7.  2.]
 [ 3. 31.  6.  0.  0.]
 [ 1.  0. 83.  1.  2.]
 [ 0.  1.  4. 51.  4.]
 [ 5.  0.  0.  0. 34.]]
0.84
[[51.  0.  0.  6.  2.]
 [ 6. 45.  9.  0.  1.]
 [ 1.  2. 88.  2.  8.]
 [ 0.  2.  4. 38.  0.]
 [ 2.  0.  6.  0. 27.]]
35
0.8033333333333333
[[57.  2.  0. 10.  2.]
 [ 1. 32.  9.  2.  0.]
 [ 0.  5. 84.  1.  2.]
 [ 8.  1.  6. 34.  3.]
 [ 6.  0.  1.  0. 34.]]
0.8066666666666666
[[60.  1.  0.  7.  4.]
 [ 3. 34.  7.  3.  0.]
 [ 0.  8. 73.  7.  5.]
 [ 4.  1. 10. 34.  1.]
 [ 5.  0.  0.  0. 33.]]
36
0.8266666666666667
[[44.  6.  0.  7.  1.]
 [ 2. 40.  4.  1.  1.]
 [ 0.  8. 74.  1.  5.]
 [ 5.  3.  2. 43.  1.]
 [ 4.  0.  1.  0. 47.]]
0.8033333333333333
[[44.  2.  0.  5.  6.]
 [ 4. 34.  4.  6.  0.]
 [ 0.  7. 81.  6.  4.]


In [17]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.8266666666666667, 0.87, 0.84, 0.8566666666666667, 0.82, 0.85, 0.85, 0.8466666666666667, 0.8166666666666667, 0.81, 0.8, 0.78, 0.8, 0.8366666666666667, 0.8233333333333334, 0.7766666666666666, 0.8333333333333334, 0.8333333333333334, 0.8033333333333333, 0.84, 0.8233333333333334, 0.7866666666666666, 0.8066666666666666, 0.8533333333333334, 0.8333333333333334, 0.8366666666666667, 0.8266666666666667, 0.8433333333333334, 0.8633333333333333, 0.81, 0.8166666666666667, 0.8466666666666667, 0.8166666666666667, 0.84, 0.8066666666666666, 0.8033333333333333, 0.7933333333333333, 0.8633333333333333, 0.8433333333333334, 0.7933333333333333]
0.8254999999999999
0.023541334805920508
[0.8175767902230762, 0.8610116546995122, 0.8258688885108441, 0.8581406575427772, 0.8081020632154006, 0.8384971068165179, 0.8431450967845144, 0.8384611541552228, 0.8015416712564967, 0.7942971887229145, 0.7824034792368126, 0.777434974952299, 0.7961735654713863, 0.8296347224945355, 0.8043561358150176, 0.7721639247418393, 0.8345354

## scalar = 10

In [18]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10/'
    file_data = rountine + 'all_data_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.5166666666666667, 0.8733333333333333, 0.86, 0.85, 0.8566666666666667, 0.86, 0.85, 0.8566666666666667, 0.86, 0.85, 0.8566666666666667, 0.86, 0.85, 0.8566666666666667, 0.86]
[[53.  2.  0.  2.  1.]
 [ 1. 37.  5.  1.  0.]
 [ 2.  2. 76.  5.  3.]
 [ 1.  4.  3. 54.  3.]
 [ 0.  0.  6.  1. 38.]]
[0.56, 0.8633333333333333, 0.85, 0.8533333333333334, 0.8566666666666667, 0.85, 0.8533333333333334, 0.8566666666666667, 0.85, 0.8533333333333334, 0.8566666666666667, 0.85, 0.8533333333333334, 0.8566666666666667, 0.85]
[[51.  2.  0.  3.  1.]
 [ 1. 47. 10.  0.  0.]
 [ 1.  4. 77. 10.  2.]
 [ 2.  2.  1. 50.  0.]
 [ 2.  0.  4.  0. 30.]]
2
[0.6533333333333333, 0.85, 0.8566666666666667, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85]
[[46.  3.  0.  7.  1.]
 [ 1. 41.  7.  0.  0.]
 [ 2.  3. 87.  2.  2.]
 [ 5.  0.  5. 44.  1.]
 [ 5.  1.  0.  0. 37.]]
[0.6366666666666667, 0.8033333333333333, 0.8133333333333334, 0.8133333333333334, 0.8133333333333334, 0.8133333333333334, 0.81333333333333

[0.5566666666666666, 0.8266666666666667, 0.82, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]
[[50.  6.  0.  4.  3.]
 [ 2. 46.  7.  0.  0.]
 [ 3.  4. 75.  4.  2.]
 [ 5.  1.  7. 33.  0.]
 [ 6.  0.  5.  1. 36.]]
[0.55, 0.8033333333333333, 0.8, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666]
[[49.  3.  0.  7.  2.]
 [ 3. 40.  8.  1.  0.]
 [ 1.  3. 66.  6.  7.]
 [ 4.  1.  4. 47.  2.]
 [ 5.  0.  4.  0. 37.]]
14
[0.61, 0.8433333333333334, 0.8466666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667, 0.8566666666666667]
[[50.  3.  0.  7.  4.]
 [ 0. 57.  3.  0.  0.]
 [ 0.  5. 68.  1.  2.]
 [ 4.  4.  2. 43.  2.]
 [ 3.  

[0.5533333333333333, 0.8866666666666667, 0.8733333333333333, 0.8666666666666667, 0.86, 0.8666666666666667, 0.86, 0.8666666666666667, 0.86, 0.8666666666666667, 0.86, 0.8666666666666667, 0.86, 0.8666666666666667, 0.86]
[[67.  1.  0.  6.  0.]
 [ 3. 40.  7.  0.  0.]
 [ 2.  5. 61.  0.  4.]
 [ 0.  1.  4. 54.  4.]
 [ 2.  0.  1.  2. 36.]]
[0.5866666666666667, 0.82, 0.7966666666666666, 0.81, 0.8133333333333334, 0.81, 0.8133333333333334, 0.81, 0.8133333333333334, 0.81, 0.8133333333333334, 0.81, 0.8133333333333334, 0.81, 0.8133333333333334]
[[52.  2.  0.  5.  2.]
 [ 0. 44. 11.  0.  0.]
 [ 0.  4. 79.  1.  3.]
 [12.  1.  8. 34.  2.]
 [ 2.  1.  1.  1. 35.]]
26
[0.38333333333333336, 0.8366666666666667, 0.8066666666666666, 0.8, 0.7966666666666666, 0.8, 0.7966666666666666, 0.8, 0.7966666666666666, 0.8, 0.7966666666666666, 0.8, 0.7966666666666666, 0.8, 0.7966666666666666]
[[49.  3.  0.  2.  3.]
 [ 0. 44.  3.  2.  0.]
 [ 1.  7. 59. 10.  3.]
 [ 2.  2.  8. 47.  4.]
 [ 6.  0.  4.  1. 40.]]
[0.41333333333333

[0.66, 0.8066666666666666, 0.8133333333333334, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81]
[[44.  4.  0.  4.  4.]
 [ 1. 42.  2.  1.  1.]
 [ 1.  8. 78.  2.  4.]
 [ 3.  4.  4. 43.  4.]
 [ 2.  1.  4.  3. 36.]]
[0.6066666666666667, 0.81, 0.8166666666666667, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81]
[[51.  5.  0.  9.  4.]
 [ 0. 39.  3.  1.  0.]
 [ 0.  5. 66.  5.  2.]
 [ 1.  3.  5. 53.  1.]
 [ 7.  0.  3.  3. 34.]]
39
[0.68, 0.8533333333333334, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85]
[[46.  2.  0.  5.  1.]
 [ 4. 39.  5.  0.  0.]
 [ 0.  5. 77.  2.  3.]
 [ 3.  1.  6. 49.  3.]
 [ 3.  0.  2.  0. 44.]]
[0.6466666666666666, 0.8033333333333333, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666]
[[54.

In [19]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.8633333333333333, 0.8133333333333334, 0.8066666666666666, 0.7833333333333333, 0.8266666666666667, 0.82, 0.78, 0.79, 0.7766666666666666, 0.8033333333333333, 0.8033333333333333, 0.85, 0.8033333333333333, 0.8133333333333334, 0.7933333333333333, 0.8233333333333334, 0.8333333333333334, 0.7933333333333333, 0.7833333333333333, 0.7866666666666666, 0.8166666666666667, 0.81, 0.7966666666666666, 0.7733333333333333, 0.82, 0.8366666666666667, 0.78, 0.7933333333333333, 0.8033333333333333, 0.7866666666666666, 0.8433333333333334, 0.7966666666666666, 0.7433333333333333, 0.81, 0.83, 0.8433333333333334, 0.8266666666666667, 0.8166666666666667, 0.8033333333333333, 0.8066666666666666]
0.8070833333333333
0.023489802752116363
[0.8622255565546293, 0.8004464436957708, 0.8033466998272584, 0.7786721752560947, 0.8209265212232978, 0.812988870489962, 0.7678842503586034, 0.7915334461912839, 0.7769985841686086, 0.802379824157659, 0.8033761582620278, 0.8478942406468846, 0.8032526523813044, 0.8051636318387632, 0.7935

## s (-0.25)

In [22]:
k = 5
s = -1/(k-1)
C_num = 3
rep = 40

In [23]:
s

-0.25

In [24]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [25]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [26]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2/'
    file_data = rountine + 'all_data_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
can not calculate
0.7633333333333333
[[ 0.  0.  0. 32. 16.]
 [ 0. 35.  6.  0.  0.]
 [ 0.  5. 81.  0.  0.]
 [ 0. 12.  0. 47.  0.]
 [ 0.  0.  0.  0. 66.]]
0.7966666666666666
[[ 0.  0.  0. 29. 12.]
 [ 0. 42.  1.  0.  0.]
 [ 0.  2. 97.  0.  0.]
 [ 0. 17.  0. 39.  0.]
 [ 0.  0.  0.  0. 61.]]
2
can not calculate
3
0.78
[[ 0.  0.  0. 26. 15.]
 [ 0. 59.  0.  0.  0.]
 [ 0. 13. 68.  0.  0.]
 [ 0. 12.  0. 39.  0.]
 [ 0.  0.  0.  0. 68.]]
0.8
[[ 0.  0.  0. 25.  8.]
 [ 0. 54.  0.  0.  0.]
 [ 0. 13. 72.  0.  0.]
 [ 0. 15.  0. 44.  0.]
 [ 0.  0.  0.  0. 69.]]
4
0.7333333333333333
[[ 0.  0.  0. 35. 20.]
 [ 0. 44.  0.  0.  0.]
 [ 0.  8. 71.  0.  0.]
 [ 0. 17.  0. 53.  0.]
 [ 0.  0.  0.  0. 52.]]
0.75
[[ 0.  0.  0. 21. 17.]
 [ 0. 46.  0.  0.  0.]
 [ 0. 18. 67.  0.  0.]
 [ 0. 19.  0. 46.  0.]
 [ 0.  0.  0.  0. 66.]]
5
0.7933333333333333
[[ 0.  0.  0. 27.  9.]
 [ 0. 58.  0.  0.  0.]
 [ 0. 15. 74.  0.  0.]
 [ 0. 11.  0. 37.  0.]
 [ 0.  0.  0.  0. 69.]]
0.7833333333333333
[[ 0.  0.  0. 31. 12.]
 [ 0. 40. 

D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


0.8033333333333333
[[ 0.  0.  0. 28. 10.]
 [ 0. 51.  0.  0.  0.]
 [ 0. 13. 80.  0.  0.]
 [ 0.  8.  0. 48.  0.]
 [ 0.  0.  0.  0. 62.]]
0.8333333333333334
[[ 0.  0.  0. 23.  9.]
 [ 0. 59.  0.  0.  0.]
 [ 0. 10. 71.  0.  0.]
 [ 0. 13.  0. 59.  0.]
 [ 0.  0.  0.  0. 56.]]
14
can not calculate
15
can not calculate
16
0.76
[[ 0.  0.  0. 24. 16.]
 [ 0. 48.  0.  0.  0.]
 [ 0. 12. 80.  0.  0.]
 [ 0. 20.  0. 39.  0.]
 [ 0.  0.  0.  0. 61.]]
0.8166666666666667
[[ 0.  0.  0. 35. 10.]
 [ 0. 51.  0.  0.  0.]
 [ 0. 14. 72.  0.  0.]
 [ 0. 10.  0. 43.  0.]
 [ 0.  0.  0.  0. 65.]]
17
0.6933333333333334
[[ 3.  0.  0. 27. 12.]
 [ 0.  0. 38. 14.  0.]
 [ 0.  0. 86.  0.  0.]
 [ 0.  0.  1. 57.  0.]
 [ 0.  0.  0.  0. 62.]]
0.7266666666666667
[[ 1.  0.  0. 30. 10.]
 [ 0.  0. 33. 11.  0.]
 [ 0.  0. 87.  0.  0.]
 [ 0.  0.  1. 60.  0.]
 [ 0.  0.  0.  0. 67.]]
18
0.81
[[ 0.  0.  0. 31.  9.]
 [ 0. 45.  0.  0.  0.]
 [ 0.  5. 78.  0.  0.]
 [ 0. 12.  0. 55.  0.]
 [ 0.  0.  0.  0. 65.]]
0.7666666666666667
[[ 0.  0.  0.

In [27]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.8, 0.75, 0.7833333333333333, 0.8066666666666666, 0.7966666666666666, 0.7466666666666667, 0.7433333333333333, 0.8333333333333334, 0.8166666666666667, 0.7266666666666667, 0.7666666666666667, 0.81, 0.77, 0.8066666666666666, 0.7833333333333333, 0.83, 0.8333333333333334, 0.6633333333333333, 0.81, 0.8166666666666667, 0.7733333333333333, 0.7933333333333333, 0.75, 0.77, 0.78, 0.77, 0.79, 0.77, 0.79, 0.8233333333333334, 0.7633333333333333]
0.7827956989247311
0.035206475192487624
[0.6981906470384056, 0.6765248133048493, 0.6653541097702842, 0.7642084325690492, 0.6718214050994822, 0.6396010860484546, 0.6346185223623239, 0.7527987306881585, 0.7820857184676594, 0.5892354524809449, 0.726420859517668, 0.7699657767462658, 0.7079599053154844, 0.6946647242946573, 0.7054480642347247, 0.6993918975242435, 0.7659290768774207, 0.4899142226217029, 0.7640969639392644, 0.7700244065379743, 0.7145835265662398, 0.7169559604338207, 0.6383205498919067, 0.6671519923642564, 0.7220550678962224, 0.7506491278417704, 0.

### s5

In [28]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5/'
    file_data = rountine + 'all_data_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.8333333333333334
[[70.  3.  0.  6.  4.]
 [ 1. 39.  6.  2.  0.]
 [ 0.  0. 77.  2.  1.]
 [ 6.  4.  2. 38.  0.]
 [11.  0.  2.  0. 26.]]
0.8033333333333333
[[35.  1.  0.  9.  2.]
 [ 1. 31.  6.  5.  0.]
 [ 1.  5. 79.  2.  2.]
 [ 9.  6.  3. 42.  2.]
 [ 7.  0.  4.  1. 47.]]
2
0.8533333333333334
[[53.  3.  0.  9.  5.]
 [ 0. 30. 10.  0.  0.]
 [ 1.  0. 72.  0.  6.]
 [ 1.  1.  1. 53.  3.]
 [ 2.  0.  2.  0. 48.]]
0.8233333333333334
[[43.  1.  0.  9.  7.]
 [ 2. 35. 15.  0.  0.]
 [ 0.  1. 84.  0.  3.]
 [ 4.  3.  3. 50.  4.]
 [ 4.  0.  0.  0. 32.]]
3
0.81
[[32.  6.  0. 11.  7.]
 [ 1. 48.  5.  1.  1.]
 [ 0.  5. 81.  1.  3.]
 [ 0.  4.  4. 48.  2.]
 [ 5.  0.  1.  0. 34.]]
0.85
[[ 34.   2.   0.  12.   3.]
 [  0.  34.   6.   1.   0.]
 [  0.   8. 101.   0.   6.]
 [  0.   1.   3.  50.   1.]
 [  4.   0.   0.   0.  34.]]
4
0.8266666666666667
[[47.  1.  0.  9.  5.]
 [ 0. 38.  9.  0.  0.]
 [ 0.  7. 77.  2.  3.]
 [ 8.  3.  1. 48.  3.]
 [ 1.  0.  0.  0. 38.]]
0.8433333333333334
[[44.  0.  0.  9.  4.]
 [ 2. 35

0.8233333333333334
[[59.  6.  0.  8.  1.]
 [ 2. 30.  5.  3.  0.]
 [ 1.  1. 81.  2.  2.]
 [ 8.  2.  2. 44.  4.]
 [ 6.  0.  0.  0. 33.]]
0.8533333333333334
[[53.  1.  0.  3.  2.]
 [ 3. 49.  9.  0.  0.]
 [ 2.  4. 88.  1.  6.]
 [ 1.  4.  1. 37.  1.]
 [ 3.  0.  6.  0. 26.]]
35
0.8033333333333333
[[57.  2.  0. 10.  2.]
 [ 1. 30. 13.  0.  0.]
 [ 1.  2. 88.  0.  1.]
 [ 8.  1.  8. 33.  2.]
 [ 6.  0.  1.  1. 33.]]
0.8066666666666666
[[60.  1.  0.  8.  3.]
 [ 3. 31. 13.  0.  0.]
 [ 0.  4. 83.  1.  5.]
 [ 4.  1. 10. 35.  0.]
 [ 5.  0.  0.  0. 33.]]
36
0.7966666666666666
[[41.  6.  0. 11.  0.]
 [ 2. 43.  3.  0.  0.]
 [ 0. 10. 73.  0.  5.]
 [ 8.  3.  6. 36.  1.]
 [ 5.  0.  1.  0. 46.]]
0.7966666666666666
[[41.  2.  0.  8.  6.]
 [ 3. 40.  4.  1.  0.]
 [ 1.  7. 83.  4.  3.]
 [ 7.  1. 10. 34.  4.]
 [ 2.  0.  1.  0. 38.]]
37
0.77
[[46.  2.  0.  8.  1.]
 [ 0. 23. 22.  0.  0.]
 [ 2.  1. 86.  0.  1.]
 [ 9.  5.  7. 43.  1.]
 [10.  0.  0.  0. 33.]]
0.79
[[38.  3.  0.  6.  2.]
 [ 2. 28. 20.  0.  0.]
 [ 3.  0.

In [29]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.8033333333333333, 0.8233333333333334, 0.85, 0.8433333333333334, 0.7766666666666666, 0.8133333333333334, 0.8333333333333334, 0.81, 0.81, 0.77, 0.8, 0.79, 0.82, 0.76, 0.7466666666666667, 0.8166666666666667, 0.8266666666666667, 0.8266666666666667, 0.7666666666666667, 0.79, 0.79, 0.8033333333333333, 0.7833333333333333, 0.8066666666666666, 0.82, 0.7666666666666667, 0.8366666666666667, 0.83, 0.7533333333333333, 0.8366666666666667, 0.82, 0.79, 0.8533333333333334, 0.8066666666666666, 0.7966666666666666, 0.79, 0.8333333333333334, 0.83, 0.8233333333333334]
0.8063247863247861
0.026760464010759454
[0.7928917931339062, 0.8115859920356954, 0.8368094229066564, 0.8403048981385639, 0.7601764337707901, 0.7969938647055177, 0.8223801227787279, 0.8036134137046167, 0.8024952129002296, 0.7609942005658575, 0.7848033239035059, 0.7807414645719499, 0.8150807595284768, 0.740202410178474, 0.7466757034111324, 0.8151764177851135, 0.8182565257865921, 0.8147174102940035, 0.7343418051482569, 0.7827620295028914, 0.78

## scalar = 10

In [30]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10/'
    file_data = rountine + 'all_data_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.45, 0.69, 0.7933333333333333, 0.7966666666666666, 0.8, 0.7966666666666666, 0.7833333333333333, 0.7933333333333333, 0.79, 0.7933333333333333, 0.7933333333333333, 0.79, 0.7966666666666666, 0.8, 0.7833333333333333]
[[46.  2.  0.  8.  2.]
 [ 2. 32.  7.  2.  1.]
 [ 3.  3. 72.  7.  3.]
 [ 6.  4.  4. 47.  4.]
 [ 0.  0.  6.  1. 38.]]
[0.43333333333333335, 0.6466666666666666, 0.7633333333333333, 0.7766666666666666, 0.7666666666666667, 0.77, 0.7566666666666667, 0.7533333333333333, 0.7566666666666667, 0.76, 0.7566666666666667, 0.7566666666666667, 0.7466666666666667, 0.7466666666666667, 0.75]
[[32.  4.  0. 16.  5.]
 [ 3. 44. 10.  1.  0.]
 [ 0.  6. 74.  9.  5.]
 [ 5.  5.  0. 45.  0.]
 [ 2.  0.  4.  0. 30.]]
2
[0.44666666666666666, 0.66, 0.77, 0.8333333333333334, 0.8633333333333333, 0.85, 0.85, 0.85, 0.84, 0.8533333333333334, 0.8533333333333334, 0.8466666666666667, 0.8566666666666667, 0.85, 0.8466666666666667]
[[44.  3.  0.  6.  4.]
 [ 1. 40.  8.  0.  0.]
 [ 1.  4. 84.  2.  5.]
 [ 1.  0.  3. 48

[0.4533333333333333, 0.6966666666666667, 0.75, 0.7633333333333333, 0.7733333333333333, 0.7833333333333333, 0.76, 0.76, 0.78, 0.7733333333333333, 0.7733333333333333, 0.7566666666666667, 0.76, 0.7866666666666666, 0.7633333333333333]
[[46.  4.  0.  6.  7.]
 [ 2. 42. 10.  1.  0.]
 [ 3. 10. 66.  8.  1.]
 [ 4.  2.  2. 37.  1.]
 [ 4.  0.  5.  1. 38.]]
[0.47, 0.6066666666666667, 0.7066666666666667, 0.7433333333333333, 0.75, 0.7466666666666667, 0.75, 0.7633333333333333, 0.7766666666666666, 0.7366666666666667, 0.7433333333333333, 0.7466666666666667, 0.7466666666666667, 0.75, 0.7566666666666667]
[[46.  3.  0.  6.  6.]
 [ 3. 38. 11.  0.  0.]
 [ 4.  5. 62.  8.  4.]
 [ 4.  2.  2. 45.  5.]
 [ 6.  0.  3.  1. 36.]]
14
[0.54, 0.7333333333333333, 0.7766666666666666, 0.7966666666666666, 0.7933333333333333, 0.8033333333333333, 0.8033333333333333, 0.8233333333333334, 0.8233333333333334, 0.8133333333333334, 0.8, 0.7933333333333333, 0.8133333333333334, 0.8033333333333333, 0.82]
[[46.  4.  0.  9.  5.]
 [ 1. 54

[0.43666666666666665, 0.7066666666666667, 0.7533333333333333, 0.8333333333333334, 0.8366666666666667, 0.8333333333333334, 0.8266666666666667, 0.83, 0.8233333333333334, 0.8166666666666667, 0.8333333333333334, 0.82, 0.8266666666666667, 0.8333333333333334, 0.85]
[[64.  1.  0.  8.  1.]
 [ 3. 46.  1.  0.  0.]
 [ 2.  9. 57.  0.  4.]
 [ 1.  2.  3. 52.  5.]
 [ 2.  0.  2.  1. 36.]]
[0.4766666666666667, 0.69, 0.73, 0.7666666666666667, 0.77, 0.7833333333333333, 0.7866666666666666, 0.7866666666666666, 0.7733333333333333, 0.75, 0.8, 0.7866666666666666, 0.7933333333333333, 0.7933333333333333, 0.7966666666666666]
[[50.  2.  0.  7.  2.]
 [ 0. 50.  5.  0.  0.]
 [ 0. 14. 66.  1.  6.]
 [10.  1.  6. 38.  2.]
 [ 2.  1.  1.  1. 35.]]
26
[0.6033333333333334, 0.7366666666666667, 0.7766666666666666, 0.81, 0.8033333333333333, 0.8033333333333333, 0.8133333333333334, 0.81, 0.8066666666666666, 0.8133333333333334, 0.8033333333333333, 0.82, 0.79, 0.82, 0.8]
[[41.  6.  0.  5.  5.]
 [ 0. 45.  4.  0.  0.]
 [ 2.  4. 69.

[0.4766666666666667, 0.6566666666666666, 0.8066666666666666, 0.8233333333333334, 0.8166666666666667, 0.82, 0.8033333333333333, 0.8, 0.79, 0.8, 0.7933333333333333, 0.8066666666666666, 0.7866666666666666, 0.8033333333333333, 0.7833333333333333]
[[42.  4.  0. 14.  4.]
 [ 0. 41.  5.  0.  0.]
 [ 2.  3. 67.  2.  4.]
 [ 4.  1.  9. 45.  1.]
 [ 8.  0.  4.  0. 40.]]
[0.5, 0.6933333333333334, 0.74, 0.7733333333333333, 0.77, 0.78, 0.7633333333333333, 0.7433333333333333, 0.7366666666666667, 0.7333333333333333, 0.7333333333333333, 0.7433333333333333, 0.72, 0.74, 0.7266666666666667]
[[32.  9.  0. 15.  3.]
 [ 3. 47.  7.  0.  0.]
 [ 4.  8. 61.  0.  1.]
 [ 4.  3.  7. 47.  8.]
 [ 7.  0.  3.  0. 31.]]
38
[0.5166666666666667, 0.6233333333333333, 0.66, 0.69, 0.69, 0.6966666666666667, 0.7166666666666667, 0.7466666666666667, 0.7533333333333333, 0.75, 0.7466666666666667, 0.7633333333333333, 0.77, 0.7733333333333333, 0.7766666666666666]
[[49.  3.  0.  2.  2.]
 [ 1. 35.  8.  2.  1.]
 [ 1.  9. 73.  3.  7.]
 [ 5. 

In [31]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.7766666666666666, 0.8333333333333334, 0.78, 0.6966666666666667, 0.7833333333333333, 0.79, 0.7166666666666667, 0.7733333333333333, 0.7666666666666667, 0.78, 0.7166666666666667, 0.7966666666666666, 0.7766666666666666, 0.7866666666666666, 0.71, 0.7533333333333333, 0.7866666666666666, 0.7533333333333333, 0.7866666666666666, 0.73, 0.7466666666666667, 0.7966666666666666, 0.7866666666666666, 0.7233333333333334, 0.8, 0.79, 0.78, 0.76, 0.7266666666666667, 0.75, 0.7766666666666666, 0.6966666666666667, 0.72, 0.77, 0.7866666666666666, 0.75, 0.78, 0.7466666666666667, 0.7766666666666666, 0.7666666666666667]
0.7630833333333333
0.030879582430970647
[0.7786510672294679, 0.825273862822679, 0.7686748216106014, 0.6840612648221345, 0.7782939863154483, 0.7801849759252967, 0.6976545056367733, 0.7684424818902873, 0.7707587012281624, 0.7757642229584658, 0.7181875996679823, 0.7955431000702781, 0.7730786176480414, 0.7795038472979318, 0.7066304066304067, 0.743638872497379, 0.7670710920129462, 0.741331381518303

## s = -0.025

In [34]:
k = 5
s = (-1/(k-1))/10
C_num = 3
rep = 40

In [35]:
s

-0.025

In [36]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [37]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [38]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2/'
    file_data = rountine + 'all_data_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
0.7466666666666667
[[ 0.  0.  0. 32. 16.]
 [ 0. 41.  0.  0.  0.]
 [ 0. 16. 70.  0.  0.]
 [ 0. 12.  0. 47.  0.]
 [ 0.  0.  0.  0. 66.]]
0.7766666666666666
[[ 0.  0.  0. 29. 12.]
 [ 0. 43.  0.  0.  0.]
 [ 0. 14. 85.  0.  0.]
 [ 0. 17.  0. 39.  0.]
 [ 0.  0.  0.  0. 61.]]
2
0.8233333333333334
[[ 0.  0.  0. 22. 13.]
 [ 0. 40.  0.  0.  0.]
 [ 0.  6. 84.  0.  0.]
 [ 0. 12.  0. 63.  0.]
 [ 0.  0.  0.  0. 60.]]
0.7666666666666667
[[ 0.  0.  0. 31. 11.]
 [ 0. 59.  0.  0.  0.]
 [ 0.  8. 58.  0.  0.]
 [ 0. 20.  0. 47.  0.]
 [ 0.  0.  0.  0. 66.]]
3
0.78
[[ 0.  0.  0. 26. 15.]
 [ 0. 59.  0.  0.  0.]
 [ 0. 13. 68.  0.  0.]
 [ 0. 12.  0. 39.  0.]
 [ 0.  0.  0.  0. 68.]]
0.7966666666666666
[[ 0.  0.  0. 25.  8.]
 [ 0. 54.  0.  0.  0.]
 [ 0. 13. 72.  0.  0.]
 [ 0. 15.  0. 44.  0.]
 [ 0.  0.  0.  0. 69.]]
4
can not calculate
0.7566666666666667
[[ 0.  0.  0. 35. 20.]
 [ 0. 43.  1.  0.  0.]
 [ 0.  0. 79.  0.  0.]
 [ 0. 17.  0. 53.  0.]
 [ 0.  0.  0.  0. 52.]]
0.79
[[ 0.  0.  0. 21. 17.]
 [ 0. 44.  2.  

can not calculate
33
0.7933333333333333
[[ 0.  0.  0. 33.  7.]
 [ 0. 45.  0.  0.  0.]
 [ 0. 10. 91.  0.  0.]
 [ 0. 12.  0. 45.  0.]
 [ 0.  0.  0.  0. 57.]]
0.7666666666666667
[[ 0.  0.  0. 33.  8.]
 [ 0. 54.  0.  0.  0.]
 [ 0. 12. 65.  0.  0.]
 [ 0. 17.  0. 54.  0.]
 [ 0.  0.  0.  0. 57.]]
34
0.7733333333333333
[[ 0.  0.  0. 23. 14.]
 [ 0. 42.  0.  0.  0.]
 [ 0. 13. 72.  0.  0.]
 [ 0. 18.  0. 46.  0.]
 [ 0.  0.  0.  0. 72.]]
0.7633333333333333
[[ 0.  0.  0. 36.  6.]
 [ 0. 43.  0.  0.  0.]
 [ 0. 10. 71.  0.  0.]
 [ 0. 21.  0. 51.  0.]
 [ 0.  0.  0.  0. 62.]]
35
0.7933333333333333
[[ 0.  0.  0. 27. 14.]
 [ 0. 44.  0.  0.  0.]
 [ 0. 11. 62.  0.  0.]
 [ 0. 10.  0. 54.  0.]
 [ 0.  0.  0.  0. 78.]]
0.74
[[ 0.  0.  0. 38. 14.]
 [ 0. 48.  0.  0.  0.]
 [ 0. 12. 56.  0.  0.]
 [ 0. 15.  0. 54.  0.]
 [ 0.  0.  0.  0. 63.]]
36
can not calculate
0.23
[[14.  0.  0. 21.  0.]
 [22.  0.  0. 19. 11.]
 [ 4. 18.  0.  0. 73.]
 [25.  0.  0. 40.  0.]
 [26.  0.  0. 12. 15.]]
0.2733333333333333
[[13.  0.  0. 26

In [39]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.7766666666666666, 0.7666666666666667, 0.7966666666666666, 0.7566666666666667, 0.6133333333333333, 0.8066666666666666, 0.7433333333333333, 0.68, 0.8166666666666667, 0.7966666666666666, 0.6766666666666666, 0.7966666666666666, 0.76, 0.7866666666666666, 0.7533333333333333, 0.7666666666666667, 0.7066666666666667, 0.7533333333333333, 0.7666666666666667, 0.83, 0.8133333333333334, 0.8033333333333333, 0.8033333333333333, 0.7866666666666666, 0.77, 0.7866666666666666, 0.7666666666666667, 0.7633333333333333, 0.74, 0.7833333333333333, 0.8233333333333334, 0.76]
0.7671874999999999
0.045070233578949986
[0.7101996232357924, 0.6630111668391645, 0.6688041157480354, 0.6633116883116883, 0.5136625092788248, 0.73190381306379, 0.6799012501584052, 0.4816801817187325, 0.6925887119768827, 0.7620337040354264, 0.5347273010040923, 0.7297117524440939, 0.6598091041277682, 0.6770046057002579, 0.6522667972536379, 0.6581528340080972, 0.6135562181447503, 0.6751069100282024, 0.6899943538445007, 0.6886396609926021, 0.68

### s5

In [40]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5/'
    file_data = rountine + 'all_data_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.7766666666666666
[[70.  3.  0.  6.  4.]
 [ 1. 42.  5.  0.  0.]
 [ 1. 14. 54.  7.  4.]
 [ 6.  4.  1. 39.  0.]
 [ 9.  0.  2.  0. 28.]]
0.79
[[35.  1.  0.  9.  2.]
 [ 1. 34.  6.  2.  0.]
 [ 2. 22. 55.  7.  3.]
 [ 8.  6.  5. 40.  3.]
 [ 7.  0.  4.  0. 48.]]
2
0.83
[[52.  3.  0. 10.  5.]
 [ 0. 28. 12.  0.  0.]
 [ 1.  0. 72.  0.  6.]
 [ 2.  1.  4. 49.  3.]
 [ 2.  0.  2.  0. 48.]]
0.8066666666666666
[[41.  2.  0. 10.  7.]
 [ 2. 32. 18.  0.  0.]
 [ 0.  1. 84.  0.  3.]
 [ 5.  3.  4. 47.  5.]
 [ 3.  0.  0.  0. 33.]]
3
0.81
[[33.  6.  0. 10.  7.]
 [ 0. 47.  4.  3.  2.]
 [ 0.  5. 80.  2.  3.]
 [ 0.  4.  3. 49.  2.]
 [ 5.  0.  1.  0. 34.]]
0.8433333333333334
[[ 34.   1.   0.  13.   3.]
 [  1.  32.   6.   2.   0.]
 [  0.   7. 102.   0.   6.]
 [  0.   0.   4.  50.   1.]
 [  3.   0.   0.   0.  35.]]
4
0.7966666666666666
[[49.  1.  0.  9.  3.]
 [ 0. 39.  8.  0.  0.]
 [ 0. 12. 68.  5.  4.]
 [ 8.  3.  3. 46.  3.]
 [ 2.  0.  0.  0. 37.]]
0.86
[[45.  0.  0.  9.  3.]
 [ 2. 36.  4.  0.  0.]
 [ 0. 14. 61.

0.8133333333333334
[[55.  3.  0.  2.  0.]
 [ 2. 32. 11.  0.  0.]
 [ 1.  0. 76.  0.  3.]
 [ 7.  3.  7. 45.  2.]
 [15.  0.  0.  0. 36.]]
0.7766666666666666
[[46.  6.  0.  5.  2.]
 [ 2. 35. 13.  0.  0.]
 [ 0.  0. 69.  0.  2.]
 [ 9.  5. 14. 36.  1.]
 [11.  0.  0.  0. 44.]]
34
0.8133333333333334
[[58.  6.  0.  7.  3.]
 [ 2. 31.  7.  0.  0.]
 [ 1.  1. 82.  1.  2.]
 [ 4.  3. 11. 38.  4.]
 [ 4.  0.  0.  0. 35.]]
0.8466666666666667
[[53.  0.  0.  4.  2.]
 [ 4. 48.  9.  0.  0.]
 [ 1.  4. 89.  0.  7.]
 [ 1.  3.  6. 33.  1.]
 [ 2.  0.  6.  0. 27.]]
35
0.7633333333333333
[[52.  1.  0. 10.  8.]
 [ 0. 30. 13.  0.  1.]
 [ 0.  7. 83.  0.  2.]
 [ 7.  5.  6. 28.  6.]
 [ 4.  0.  1.  0. 36.]]
0.7566666666666667
[[52.  1.  0. 11.  8.]
 [ 3. 36.  8.  0.  0.]
 [ 0.  8. 79.  1.  5.]
 [ 8.  0. 12. 25.  5.]
 [ 4.  0.  0.  0. 34.]]
36
0.8166666666666667
[[41.  6.  0. 11.  0.]
 [ 2. 41.  5.  0.  0.]
 [ 0.  4. 79.  0.  5.]
 [ 7.  3.  4. 39.  1.]
 [ 6.  0.  1.  0. 45.]]
0.8
[[41.  2.  0.  8.  6.]
 [ 3. 40.  3.  2.  

In [41]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.79, 0.8066666666666666, 0.8433333333333334, 0.86, 0.78, 0.8, 0.8033333333333333, 0.8266666666666667, 0.7866666666666666, 0.7366666666666667, 0.7933333333333333, 0.7933333333333333, 0.7933333333333333, 0.81, 0.7933333333333333, 0.76, 0.8233333333333334, 0.8033333333333333, 0.82, 0.7666666666666667, 0.7966666666666666, 0.7966666666666666, 0.8033333333333333, 0.8166666666666667, 0.8066666666666666, 0.7633333333333333, 0.7766666666666666, 0.8266666666666667, 0.7866666666666666, 0.73, 0.8, 0.82, 0.7766666666666666, 0.8466666666666667, 0.7566666666666667, 0.8, 0.7566666666666667, 0.8033333333333333, 0.8233333333333334, 0.8]
0.7969166666666666
0.027369767384226463
[0.7771170608542237, 0.7931926517272234, 0.8259107284033002, 0.860701132483815, 0.7658687123238291, 0.7857719129925915, 0.7769197665443309, 0.817270174751255, 0.7728859405171866, 0.7259305064701279, 0.7834051006581494, 0.7806592071278129, 0.7827052719332557, 0.8047063542398091, 0.7737770243012004, 0.7540481506440784, 0.8213460666

## scalar = 10

In [42]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10/'
    file_data = rountine + 'all_data_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.4766666666666667, 0.66, 0.7033333333333334, 0.75, 0.7933333333333333, 0.7833333333333333, 0.77, 0.7766666666666666, 0.7766666666666666, 0.7566666666666667, 0.77, 0.7633333333333333, 0.77, 0.76, 0.7666666666666667]
[[46.  2.  0.  8.  2.]
 [ 2. 32.  9.  1.  0.]
 [ 3.  3. 71.  8.  3.]
 [ 7.  5.  6. 43.  4.]
 [ 1.  0.  5.  1. 38.]]
[0.4266666666666667, 0.6733333333333333, 0.7133333333333334, 0.73, 0.7633333333333333, 0.79, 0.76, 0.78, 0.7633333333333333, 0.74, 0.76, 0.78, 0.7633333333333333, 0.7766666666666666, 0.7633333333333333]
[[44.  2.  0.  6.  5.]
 [ 2. 42. 11.  3.  0.]
 [ 2.  5. 72. 10.  5.]
 [ 2.  5.  0. 42.  6.]
 [ 3.  0.  4.  0. 29.]]
2
[0.56, 0.7966666666666666, 0.8533333333333334, 0.8266666666666667, 0.8133333333333334, 0.8233333333333334, 0.8366666666666667, 0.8466666666666667, 0.8533333333333334, 0.8366666666666667, 0.8466666666666667, 0.8433333333333334, 0.85, 0.8433333333333334, 0.84]
[[42.  4.  0.  7.  4.]
 [ 1. 40.  8.  0.  0.]
 [ 2.  2. 87.  1.  4.]
 [ 3.  0.  5. 45

[0.48, 0.6766666666666666, 0.7266666666666667, 0.7733333333333333, 0.76, 0.7466666666666667, 0.7233333333333334, 0.7333333333333333, 0.7333333333333333, 0.7266666666666667, 0.7233333333333334, 0.7066666666666667, 0.71, 0.6866666666666666, 0.6966666666666667]
[[33. 11.  0.  8. 11.]
 [ 1. 40. 11.  3.  0.]
 [ 6. 10. 65.  6.  1.]
 [ 5.  2.  2. 34.  3.]
 [ 4.  0.  6.  1. 37.]]
[0.5, 0.68, 0.7266666666666667, 0.6933333333333334, 0.7433333333333333, 0.75, 0.7166666666666667, 0.7166666666666667, 0.72, 0.7166666666666667, 0.67, 0.7, 0.68, 0.6666666666666666, 0.68]
[[29.  8.  0. 11. 13.]
 [ 3. 34. 14.  1.  0.]
 [ 3.  4. 63.  8.  5.]
 [ 4.  2.  2. 42.  8.]
 [ 6.  0.  3.  1. 36.]]
14
[0.4033333333333333, 0.6633333333333333, 0.7666666666666667, 0.7933333333333333, 0.7966666666666666, 0.7933333333333333, 0.79, 0.7833333333333333, 0.7933333333333333, 0.7866666666666666, 0.7933333333333333, 0.79, 0.7966666666666666, 0.7933333333333333, 0.7966666666666666]
[[42.  4.  0. 11.  7.]
 [ 0. 56.  4.  0.  0.]


[0.4266666666666667, 0.59, 0.6933333333333334, 0.7166666666666667, 0.7433333333333333, 0.74, 0.7433333333333333, 0.76, 0.7666666666666667, 0.7633333333333333, 0.7766666666666666, 0.7733333333333333, 0.7866666666666666, 0.7833333333333333, 0.7833333333333333]
[[50.  5.  0. 14.  5.]
 [ 2. 40.  6.  0.  2.]
 [ 0.  7. 58.  1.  6.]
 [ 5.  0.  1. 53.  4.]
 [ 4.  0.  2.  1. 34.]]
[0.5, 0.63, 0.69, 0.68, 0.6933333333333334, 0.7033333333333334, 0.6766666666666666, 0.73, 0.6966666666666667, 0.7233333333333334, 0.7333333333333333, 0.7266666666666667, 0.7566666666666667, 0.77, 0.74]
[[34.  4.  0. 19.  4.]
 [ 2. 44.  8.  1.  0.]
 [ 1. 10. 69.  2.  5.]
 [ 2.  2.  4. 41.  8.]
 [ 3.  1.  1.  1. 34.]]
26
[0.6, 0.7466666666666667, 0.7733333333333333, 0.7766666666666666, 0.77, 0.7866666666666666, 0.7866666666666666, 0.8, 0.8, 0.7966666666666666, 0.8066666666666666, 0.7966666666666666, 0.8, 0.7966666666666666, 0.8066666666666666]
[[42.  3.  0.  5.  7.]
 [ 0. 44.  5.  0.  0.]
 [ 0.  5. 67.  4.  4.]
 [ 7.  0

[0.42, 0.6566666666666666, 0.7166666666666667, 0.7366666666666667, 0.75, 0.76, 0.77, 0.76, 0.7766666666666666, 0.77, 0.7833333333333333, 0.78, 0.7733333333333333, 0.77, 0.75]
[[28.  8.  0. 18. 10.]
 [ 2. 42.  2.  0.  0.]
 [ 2.  6. 61.  4.  5.]
 [ 2.  4.  2. 50.  2.]
 [ 4.  0.  3.  1. 44.]]
[0.42, 0.6233333333333333, 0.66, 0.6766666666666666, 0.7, 0.7066666666666667, 0.7233333333333334, 0.73, 0.7266666666666667, 0.7333333333333333, 0.7333333333333333, 0.7433333333333333, 0.74, 0.7133333333333334, 0.73]
[[26. 10.  0. 16.  7.]
 [ 6. 44.  7.  0.  0.]
 [ 0.  5. 64.  2.  3.]
 [ 2.  7.  2. 49.  9.]
 [ 2.  0.  3.  0. 36.]]
38
[0.5166666666666667, 0.69, 0.72, 0.7266666666666667, 0.7233333333333334, 0.7066666666666667, 0.69, 0.69, 0.7166666666666667, 0.72, 0.72, 0.69, 0.7066666666666667, 0.6833333333333333, 0.68]
[[47.  2.  0.  3.  4.]
 [ 3. 22. 19.  2.  1.]
 [ 3. 11. 57. 12. 10.]
 [ 6.  3.  6. 42.  1.]
 [ 5.  0.  4.  1. 36.]]
[0.51, 0.65, 0.69, 0.7, 0.7266666666666667, 0.7066666666666667, 0.713

In [43]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.79, 0.83, 0.7933333333333333, 0.76, 0.7133333333333334, 0.78, 0.7166666666666667, 0.7233333333333334, 0.7066666666666667, 0.7766666666666666, 0.6666666666666666, 0.7266666666666667, 0.75, 0.76, 0.6933333333333334, 0.7233333333333334, 0.7733333333333333, 0.7266666666666667, 0.7633333333333333, 0.72, 0.7633333333333333, 0.7566666666666667, 0.7733333333333333, 0.7333333333333333, 0.77, 0.7866666666666666, 0.75, 0.7033333333333334, 0.73, 0.7366666666666667, 0.75, 0.73, 0.78, 0.7166666666666667, 0.7566666666666667, 0.7866666666666666, 0.7433333333333333, 0.7266666666666667, 0.7366666666666667, 0.69]
0.7453333333333333
0.03284475266732538
[0.7849807356656672, 0.8163056323910685, 0.7868653345963261, 0.7539198375794445, 0.7107306657613878, 0.7680127702131901, 0.7015183980275592, 0.7251630654321415, 0.7016810741968359, 0.7777980630001822, 0.6644542258372046, 0.7275305492735178, 0.7461433848907687, 0.7547880608254469, 0.6937261333148893, 0.7086353207190392, 0.754305977710233, 0.71455132687377